In [1]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata
import warnings
warnings.filterwarnings('ignore')

In [2]:
#валовая прибыль (в 1с - настройка 9-7 для pandas (abcxyz))
df = pd.read_csv('1C data 01.07.23 - 31.07.23.txt', sep='\t', low_memory=False, dtype='str', encoding='UTF-8')

In [3]:
#прайс (в 1с - настройка)
price = pd.read_csv('08_price_01_08_2023_NO_ASIN_FOR_ALL.txt', sep='\t')

In [4]:
df = df.drop('Unnamed: 0', axis=1)
# 1. Удалить первые 9 строк
df = df.iloc[10:]
# 2. Удалить строки где все столбцы пустые
df = df.dropna(how='all')


In [5]:
# 4. Переименовать столбцы
df = df.rename(columns={'Валовая прибыль' : 'region',
                        'Unnamed: 2' : 'counterparty', 
                        'Unnamed: 3' : 'revenue', 
                        'Unnamed: 4' : 'detal', 
                        'Unnamed: 5' : 'brand', 
                        'Unnamed: 6' : 'nomenclature', 
                        'Unnamed: 7' : 'code1c',
                        'Unnamed: 8' : 'oem',
                        'Unnamed: 9' : 'quantity',
                        'Unnamed: 10' : 'sales'})

In [6]:
company_name = ['ООО "Москворечье Трейдинг"',
                'ООО "Москворечье-Воронеж"',
               'ООО "Москворечье-Казань"',
               'ООО "Москворечье Юг"',
               'ООО "Москворечье-Нижний"',
               'ООО "МСК-КУБАНЬ"']

ООО "Москворечье-Воронеж"
ООО "Москворечье-Казань"
ООО "Москворечье Юг"
ООО "Москворечье-Нижний"
ООО "МСК-КУБАНЬ"

In [7]:
df = df.query('counterparty == @company_name')

In [8]:
df = df.query('revenue == "Выручка (VMI)"')

In [9]:
df

,region,counterparty,revenue,detal,brand,nomenclature,code1c,oem,quantity,sales
266,Москва г,"ООО ""Москворечье Трейдинг""",Выручка (VMI),Фильтр воздушный (2015),AMD,Фильтр воздушный 28113-H8100/AMD.FA789 AMD,508240,AMD.FA789,"121,000","28 072,00"
276,Москва г,"ООО ""Москворечье Трейдинг""",Выручка (VMI),Фильтр салонный пылевой (2015),AMD,Фильтр салонный 97133-4L000/AMD.FC167 AMD,50988,AMD.FC167,"117,000","12 870,00"
282,Москва г,"ООО ""Москворечье Трейдинг""",Выручка (VMI),Фильтр воздушный (2015),AMD,Фильтр воздушный 28113-H8100/AMD.FA789 AMD,508240,AMD.FA789,"113,000","26 216,00"
399,Москва г,"ООО ""Москворечье Трейдинг""",Выручка (VMI),Фильтр воздушный (2015),AMD,Фильтр воздушный 28113-H8100/AMD.FA789 AMD,508240,AMD.FA789,"98,000","22 736,00"
404,Москва г,"ООО ""Москворечье Трейдинг""",Выручка (VMI),Фильтр воздушный (2015),AMD,Фильтр воздушный 28113-1R100/AMD.FA91 AMD,51575,AMD.FA91,"97,000","17 169,00"
...,...,...,...,...,...,...,...,...,...,...
107691,Татарстан Респ,"ООО ""Москворечье-Казань""",Выручка (VMI),Электробензонасос (мотор) (2015),AMD,Электробензонасос 31111-1R000/AMD.FP143 AMD,51125,AMD.FP143,"1,000","689,00"
107697,Татарстан Респ,"ООО ""Москворечье-Казань""",Выручка (VMI),Электробензонасос (мотор) (2015),AMD,Электробензонасос 31111-25000/AMD.FP127 AMD,40323,AMD.FP127,"1,000","683,00"
107702,Татарстан Респ,"ООО ""Москворечье-Казань""",Выручка (VMI),Электробензонасос (мотор) (2015),AMD,Электробензонасос 31111-2P900/AMD.FP211 AMD,512467,AMD.FP211,"1,000","1 647,00"
107706,Татарстан Респ,"ООО ""Москворечье-Казань""",Выручка (VMI),Электробензонасос (мотор) (2015),AMD,Электробензонасос 31119-C9000/AMD.FP173 AMD,509474,AMD.FP173,"1,000","1 002,00"


In [10]:
#долго
# 5. замена точек на запятые
df = df.stack().str.replace(',','.').unstack()
# 6. Убрать пробелы в количестве и продажах
df['quantity'] = df['quantity'].str.replace(r'\s+','',regex=True)
df['sales'] = df['sales'].str.replace(r'\s+','',regex=True)

In [11]:
df['quantity'] = np.float32(df['quantity'])
df['sales'] = np.float32(df['sales'])

In [12]:
df['quantity'] = np.int32(df['quantity'])

In [13]:
#Долго
# 8. Создание уникального кода
df['uniq'] = df.agg('{0[code1c]}{0[oem]}'.format, axis=1)
# 9. Удаление пробелов в уникальном коде
df['uniq'] = df['uniq'].replace(r'\s+', '', regex=True)

In [14]:
df

,region,counterparty,revenue,detal,brand,nomenclature,code1c,oem,quantity,sales,uniq
266,Москва г,"ООО ""Москворечье Трейдинг""",Выручка (VMI),Фильтр воздушный (2015),AMD,Фильтр воздушный 28113-H8100/AMD.FA789 AMD,508240,AMD.FA789,121,28072.0,508240AMD.FA789
276,Москва г,"ООО ""Москворечье Трейдинг""",Выручка (VMI),Фильтр салонный пылевой (2015),AMD,Фильтр салонный 97133-4L000/AMD.FC167 AMD,50988,AMD.FC167,117,12870.0,50988AMD.FC167
282,Москва г,"ООО ""Москворечье Трейдинг""",Выручка (VMI),Фильтр воздушный (2015),AMD,Фильтр воздушный 28113-H8100/AMD.FA789 AMD,508240,AMD.FA789,113,26216.0,508240AMD.FA789
399,Москва г,"ООО ""Москворечье Трейдинг""",Выручка (VMI),Фильтр воздушный (2015),AMD,Фильтр воздушный 28113-H8100/AMD.FA789 AMD,508240,AMD.FA789,98,22736.0,508240AMD.FA789
404,Москва г,"ООО ""Москворечье Трейдинг""",Выручка (VMI),Фильтр воздушный (2015),AMD,Фильтр воздушный 28113-1R100/AMD.FA91 AMD,51575,AMD.FA91,97,17169.0,51575AMD.FA91
...,...,...,...,...,...,...,...,...,...,...,...
107691,Татарстан Респ,"ООО ""Москворечье-Казань""",Выручка (VMI),Электробензонасос (мотор) (2015),AMD,Электробензонасос 31111-1R000/AMD.FP143 AMD,51125,AMD.FP143,1,689.0,51125AMD.FP143
107697,Татарстан Респ,"ООО ""Москворечье-Казань""",Выручка (VMI),Электробензонасос (мотор) (2015),AMD,Электробензонасос 31111-25000/AMD.FP127 AMD,40323,AMD.FP127,1,683.0,40323AMD.FP127
107702,Татарстан Респ,"ООО ""Москворечье-Казань""",Выручка (VMI),Электробензонасос (мотор) (2015),AMD,Электробензонасос 31111-2P900/AMD.FP211 AMD,512467,AMD.FP211,1,1647.0,512467AMD.FP211
107706,Татарстан Респ,"ООО ""Москворечье-Казань""",Выручка (VMI),Электробензонасос (мотор) (2015),AMD,Электробензонасос 31119-C9000/AMD.FP173 AMD,509474,AMD.FP173,1,1002.0,509474AMD.FP173


In [15]:
df = df.groupby(['uniq', 'code1c', 'oem', 'detal', 'brand', 'nomenclature'], as_index=False).agg({'quantity' : 'sum', 'sales' : 'sum'})

In [16]:
df

,uniq,code1c,oem,detal,brand,nomenclature,quantity,sales
0,000495AMD.EL521,000495,AMD.EL521,Реле (2015),AMD,Реле указателя поворотов 96312545/AMD.EL521 AMD,41,6847.0
1,000568AMD.EL381,000568,AMD.EL381,Стартер (части) (2015),AMD,Бендикс стартера 10475974/AMD.EL381 (9 зубов) AMD,2,782.0
2,00198AMD.EL133,00198,AMD.EL133,Генератор (2015),AMD,Генератор 96252551/AMD.EL133 (POLY-V5) AMD,4,24816.0
3,00200AMD.EL135,00200,AMD.EL135,Генератор (2015),AMD,Генератор 96567255/AMD.EL135 (POLY-V3) AMD,22,111100.0
4,00235AMD.EL218,00235,AMD.EL218,Стартер (2015),AMD,Стартер 36100-22855/AMD.EL218 (8T) AMD,9,28998.0
...,...,...,...,...,...,...,...,...
1710,6771AMD.BH29,6771,AMD.BH29,Шланг тормозной (2015),AMD,Шланг тормозной задний 96179489/AMD.BH29 AMD,7,1883.0
1711,8938AMD.BL126,8938,AMD.BL126,Ремень приводной (2015),AMD,Ремень 4PK0850 поликлиновой. кондиционера 9771...,2,486.0
1712,8945AMD.BL147,8945,AMD.BL147,Ремень приводной (2015),AMD,Ремень 5PK0870 поликлиновой. генератора 961449...,1,167.0
1713,8966AMD.BL129,8966,AMD.BL129,Ремень приводной (2015),AMD,Ремень 4PK0885 поликлиновой. генератора 0K21A-...,3,510.0


In [17]:
df.query('code1c == "56320"')

,uniq,code1c,oem,detal,brand,nomenclature,quantity,sales
1580,56320AMD.PL11,56320,AMD.PL11,Свеча зажигания (2015),AMD,Свеча зажигания 18814-11051/BKR5ES-11/AMD.PL11...,61,4392.0


In [18]:
df.dtypes

uniq             object
code1c           object
oem              object
detal            object
brand            object
nomenclature     object
quantity          int32
sales           float32
dtype: object

In [19]:
df

,uniq,code1c,oem,detal,brand,nomenclature,quantity,sales
0,000495AMD.EL521,000495,AMD.EL521,Реле (2015),AMD,Реле указателя поворотов 96312545/AMD.EL521 AMD,41,6847.0
1,000568AMD.EL381,000568,AMD.EL381,Стартер (части) (2015),AMD,Бендикс стартера 10475974/AMD.EL381 (9 зубов) AMD,2,782.0
2,00198AMD.EL133,00198,AMD.EL133,Генератор (2015),AMD,Генератор 96252551/AMD.EL133 (POLY-V5) AMD,4,24816.0
3,00200AMD.EL135,00200,AMD.EL135,Генератор (2015),AMD,Генератор 96567255/AMD.EL135 (POLY-V3) AMD,22,111100.0
4,00235AMD.EL218,00235,AMD.EL218,Стартер (2015),AMD,Стартер 36100-22855/AMD.EL218 (8T) AMD,9,28998.0
...,...,...,...,...,...,...,...,...
1710,6771AMD.BH29,6771,AMD.BH29,Шланг тормозной (2015),AMD,Шланг тормозной задний 96179489/AMD.BH29 AMD,7,1883.0
1711,8938AMD.BL126,8938,AMD.BL126,Ремень приводной (2015),AMD,Ремень 4PK0850 поликлиновой. кондиционера 9771...,2,486.0
1712,8945AMD.BL147,8945,AMD.BL147,Ремень приводной (2015),AMD,Ремень 5PK0870 поликлиновой. генератора 961449...,1,167.0
1713,8966AMD.BL129,8966,AMD.BL129,Ремень приводной (2015),AMD,Ремень 4PK0885 поликлиновой. генератора 0K21A-...,3,510.0


In [20]:
df.quantity.sum()

31114

In [21]:
df.sales.sum()

11442678.0

In [22]:
price = price.iloc[10:]
price = price.drop('Unnamed: 0', axis=1)
price  = price.rename(columns={'Unnamed: 1' : 'code1c', 
                               'Unnamed: 2' : 'oem',
                               'Unnamed: 3' : 'art',
                               'Unnamed: 4' : 'brand',
                               'Unnamed: 5' : 'group',
                               'Unnamed: 6' : 'detal',
                               'Unnamed: 7' : 'nomenclature',
                               'Unnamed: 8' : 'zn',
                               'Unnamed: 9' : 'comment',
                               'Unnamed: 10' : 'price',
                               'Unnamed: 11' : 'new'})

In [23]:
price

,code1c,oem,art,brand,group,detal,nomenclature,zn,comment,price,new
10,525071,AMD.FC944A,04596501AB,AMD,Фильтр (2015),Фильтр салонный антибактериальный (2015),.Фильтр салонный 04596501AB/AMD.FC944A (антиба...,AMD / FCR 2018 (Склад),CHRYSLER (300 C LX) (2004-2010),"344,00",NaN
11,525080,AMD.FC957A,05058693AA,AMD,Фильтр (2015),Фильтр салонный антибактериальный (2015),.Фильтр салонный 05058693AA/AMD.FC957A (антиба...,AMD / FCR 2018 (Склад),JEEP (Patriot/Liberty MK74) (2007-2017),"254,00",NaN
12,519500,AMD.FC11A,0K2N16152X,AMD,Фильтр (2015),Фильтр салонный антибактериальный (2015),.Фильтр салонный 0K2N-161-52X/AMD.FC11A (антиб...,AMD / FCR 2018 (Склад),KIA (Spectra ИЖ) (Shuma FB),"463,00",16.08.2022 0:00:00
13,525275,AMD.FC56A,0K55361C14A,AMD,Фильтр (2015),Фильтр салонный антибактериальный (2015),.Фильтр салонный 0K553-61-C14A/AMD.FC56A (анти...,AMD / FCR 2018 (Склад),"KIA (Carnival UP, GQ)","308,00",NaN
14,525632,AMD.FC981A,1017016545,AMD,Фильтр (2015),Фильтр салонный антибактериальный (2015),.Фильтр салонный 1017016545/AMD.FC981A (антиба...,AMD / FCR 2018 (Склад),GEELY (Emgrand X7) (2018-) (CA),"220,00",NaN
...,...,...,...,...,...,...,...,...,...,...,...
4551,54258,AMD.FP159,311113K000,AMD,Электрика (2015),Электробензонасос (мотор) (2015),Электробензонасос 31111-3K000/AMD.FP159 AMD,AMD / FCR 2018 (Склад),HYUNDAI (Accent MC) (2005-) (Elantra HD) (2006...,"681,00",18.09.2013 0:00:00
4552,512468,AMD.FP212,311113L800,AMD,Электрика (2015),Электробензонасос (мотор) (2015),Электробензонасос 31111-3L800/AMD.FP212 AMD,AMD / FCR 2018 (Склад),"KIA (Optima TF) (2,0) (2,4) (2010-) (Soul AM) ...","506,00",05.06.2019 0:00:00
4553,509474,AMD.FP173,31119C9000,AMD,Электрика (2015),Электробензонасос (мотор) (2015),Электробензонасос 31119-C9000/AMD.FP173 AMD,AMD / FCR 2018 (Склад),HYUNDAI (Solaris II) (2017-) (Creta) KIA (Rio ...,"992,00",23.11.2018 0:00:00
4554,515954,AMD.FP219,8200307403,AMD,Электрика (2015),Электробензонасос (мотор) (2015),Электробензонасос 8200307403/AMD.FP219 AMD,AMD / FCR 2018 (Склад),RENAULT (Logan LS) (2004-2015) (Sandero BS0) (...,"801,00",16.11.2020 0:00:00


In [24]:
price = price.query('brand == ["AMD", "FRANCECAR"]')

In [25]:
amd = df.query('brand == "AMD"')
fcr  = df.query('brand == "FRANCECAR"')

In [26]:
fm = df.query('detal == "Фильтр масляный (2015)" and  brand == ["AMD", "FRANCECAR"]')

fv = df.query('detal == "Фильтр воздушный (2015)" and  brand == ["AMD", "FRANCECAR"]')

fsp = df.query('detal == "Фильтр салонный пылевой (2015)" and  brand == ["AMD", "FRANCECAR"]')

fsu = df.query('detal == "Фильтр салонный угольный (2015)" and  brand == ["AMD", "FRANCECAR"]')

fsa = df.query('detal == "Фильтр салонный антибактериальный (2015)" and  brand == ["AMD", "FRANCECAR"]')

ft = df.query('detal == "Фильтр топливный (2015)" and  brand == ["AMD", "FRANCECAR"]')

kf = df.query('detal == "Комплект фильтров (2015)" and  brand == ["AMD", "FRANCECAR"]')

kz = df.query('detal == "Катушка зажигания (2015)" and  brand == ["AMD", "FRANCECAR"]')

e2 = df.query('detal == ["Генератор (2015)", \
                         "Стартер (2015)", \
                         "Дроссельная заслонка (2015)", \
                         "Электробензонасос (в сборе) (2015)", \
                         "Клапан рециркуляции отработавших газов (2015)"] \
                        and  brand == ["AMD", "FRANCECAR"]')

e3 = df.query('detal == ["Датчик ABS (2015)",\
                         "Датчик бокового удара (2015)", \
                         "Датчик давления воздуха (2015)", \
                         "Датчик давления ГУР (2015)", \
                         "Датчик давления масла (2015)", \
                         "Датчик детонации (2015)", \
                         "Датчик заднего хода (2015)", \
                         "Датчик коленвала (2015)", \
                         "Датчик открывания двери (2015)", \
                         "Датчик парковки (2015)", \
                         "Датчик педали тормоза (2015)", \
                         "Датчик переключения передач (2015)", \
                         "Датчик положения дросельной заслонки (2015)", \
                         "Датчик распредвала (2015)", \
                         "Датчик расхода воздуха (2015)", \
                         "Датчик системы охлаждения (2015)", \
                         "Датчик скорости (2015)", \
                         "Датчик стоп-сигнала (2015)", \
                         "Датчик температуры воздуха (2015)", \
                         "Датчик уровня бачка стеклоомывателя (2015)", \
                         "Датчик уровня топлива (2015)", \
                         "Датчик Холла (2015)", \
                         "Датчик холостого хода (2015)", \
                         "Датчики ABS (2015)"] \
                         and  brand == ["AMD", "FRANCECAR"]')

e4 = df.query('detal == ["Бачок (2015)",\
                         "Датчик кислородный (2015)",\
                         "Форсунка инжектора (2015)",\
                         "Электробензонасос (мотор) (2015)"] \
                        and  brand == ["AMD", "FRANCECAR"]')

e5 = df.query('detal == "Свеча зажигания (2015)" and  brand == ["AMD", "FRANCECAR"]')

In [27]:
amd_q = amd
amd_s = amd.sort_values('sales', ascending=False)
fcr_q = fcr
fcr_s = fcr.sort_values('sales', ascending=False)

In [28]:
fm_q = fm
fm_s = fm.sort_values('sales', ascending=False)
fv_q = fv
fv_s = fv.sort_values('sales', ascending=False)
fsp_q = fsp
fsp_s = fsp.sort_values('sales', ascending=False)
fsu_q = fsu
fsu_s = fsu.sort_values('sales', ascending=False)
fsa_q = fsa
fsa_s = fsa.sort_values('sales', ascending=False)
ft_q = ft
ft_s = ft.sort_values('sales', ascending=False)
kf_q = kf
kf_s = kf.sort_values('sales', ascending=False)
kz_q = kz
kz_s = kz.sort_values('sales', ascending=False)
e2_q = e2
e2_s = e2.sort_values('sales', ascending=False)
e3_q = e3
e3_s = e3.sort_values('sales', ascending=False)
e4_q = e4
e4_s = e4.sort_values('sales', ascending=False)
e5_q = e5 
e5_s = e5.sort_values('sales', ascending=False)

In [29]:
def calculation_cum_quantity(q):
    all_quantity = q['quantity'].sum()
    q['cumsum_q'] = q['quantity'].cumsum()
    q['di_q'] = q.cumsum_q.divide(all_quantity).round(5) * 100
    q['di_q'] = np.float32(q['di_q'])
    one_q = q.quantity.mean()
    q['one_q'] = np.where(q['quantity'] >= one_q, '0', q['quantity'])
    q['one_q'] = np.float32(q['one_q'])
    q['index_q'] = range(1, len(q) + 1)
    q['index_q'] = np.float32(q['index_q'])

In [30]:
def categorization_quantity(q):
    count = (q['one_q'] == 0).sum()
    q['one_q'] = np.float32(q['one_q'])
    two = q.one_q.sum() / (q.one_q.count() - count)
    q['XYZ'] = np.where((q['one_q'] != 0.0) & (q['one_q'] >= two)
                     , 'Y'
                     , (np.where(q['one_q'] == 0, 'X', 'Z')))

In [31]:
def calculation_cum_sales(s):
    all_sales = s['sales'].sum()
    s['cumsum_s'] = s['sales'].cumsum()
    s['di_s'] = s.cumsum_s.divide(all_sales).round(5) * 100
    s['di_s'] = np.float32(s[['di_s']])
    one_s = s.sales.mean()
    s['one_s'] = np.where(s['sales'] >= one_s, '0', s['sales'])
    s['one_s'] = np.float32(s['one_s'])
    s['index_s'] = range(1, len(s) + 1)
    s['index_s'] = np.float32(s['index_s'])

In [32]:
def categorization_sales(s):
    count_s = (s['one_s'] == 0).sum()
    two_s = s.one_s.sum() / (s.one_s.count() - count_s)
    s['ABC'] = np.where((s['one_s'] != 0.0) & (s['one_s'] >= two_s)
                     , 'B'
                     , (np.where(s['one_s'] == 0, 'A', 'C')))

In [33]:
calculation_cum_quantity(fm_q)
calculation_cum_sales(fm_s)

calculation_cum_quantity(fv_q)
calculation_cum_sales(fv_s)

calculation_cum_quantity(fsp_q)
calculation_cum_sales(fsp_s)

calculation_cum_quantity(fsu_q)
calculation_cum_sales(fsu_s)

calculation_cum_quantity(fsa_q)
calculation_cum_sales(fsa_s)

calculation_cum_quantity(ft_q)
calculation_cum_sales(ft_s)

calculation_cum_quantity(kf_q)
calculation_cum_sales(kf_s)

calculation_cum_quantity(kz_q)
calculation_cum_sales(kz_s)

calculation_cum_quantity(e2_q)
calculation_cum_sales(e2_s)

calculation_cum_quantity(e3_q)
calculation_cum_sales(e3_s)

calculation_cum_quantity(e4_q)
calculation_cum_sales(e4_s)

calculation_cum_quantity(e5_q)
calculation_cum_sales(e5_s)

calculation_cum_quantity(amd_q)
calculation_cum_sales(amd_s)

calculation_cum_quantity(fcr_q)
calculation_cum_sales(fcr_s)

In [34]:
categorization_quantity(fm_q)
categorization_sales(fm_s)

categorization_quantity(fv_q)
categorization_sales(fv_s)

categorization_quantity(fsp_q)
categorization_sales(fsp_s)

categorization_quantity(fsu_q)
categorization_sales(fsu_s)

categorization_quantity(fsa_q)
categorization_sales(fsa_s)

categorization_quantity(ft_q)
categorization_sales(ft_s)

categorization_quantity(kf_q)
categorization_sales(kf_s)

categorization_quantity(kz_q)
categorization_sales(kz_s)

categorization_quantity(e2_q)
categorization_sales(e2_s)

categorization_quantity(e3_q)
categorization_sales(e3_s)

categorization_quantity(e4_q)
categorization_sales(e4_s)

categorization_quantity(e5_q)
categorization_sales(e5_s)

categorization_quantity(amd_q)
categorization_sales(amd_s)

categorization_quantity(fcr_q)
categorization_sales(fcr_s)

In [35]:
a = 0.7
b = 0.3

In [36]:
def merge_quantity_sales(q, s):
    if q.empty or s.empty:
        return pd.DataFrame()
    merge_result = q.merge(s[['uniq',
                     'sales', 
                     'di_s', 
                     'index_s',
                     'ABC']], how='inner', left_on='uniq', right_on='uniq')
    merge_result['ABCXYZ'] = merge_result.agg('{0[ABC]}{0[XYZ]}'.format, axis=1)
    merge_result['score'] = ((merge_result['index_q'] * a) + (merge_result['index_s'] * b))
    merge_result['total_rank'] = rankdata(merge_result.score).astype(int)
    merge_result = merge_result.sort_values(['index_s'])
    return merge_result
#нужно заменить r на merge_result

In [37]:
fm = merge_quantity_sales(fm_q, fm_s)
fv = merge_quantity_sales(fv_q, fv_s)
fsp = merge_quantity_sales(fsp_q, fsp_s)
fsu = merge_quantity_sales(fsu_q, fsu_s)
fsa = merge_quantity_sales(fsa_q, fsa_s)
ft = merge_quantity_sales(ft_q, ft_s)
kf = merge_quantity_sales(kf_q, kf_s)
kz = merge_quantity_sales(kz_q, kz_s)
e2 = merge_quantity_sales(e2_q, e2_s)
e3 = merge_quantity_sales(e3_q, e3_s)
e4 = merge_quantity_sales(e4_q, e4_s)
e5 = merge_quantity_sales(e5_q, e5_s)
amd = merge_quantity_sales(amd_q, amd_s)
fcr = merge_quantity_sales(fcr_q, fcr_s)

In [38]:
e5

,uniq,code1c,oem,detal,brand,nomenclature,quantity,sales_x,cumsum_q,di_q,one_q,index_q,XYZ,sales_y,di_s,index_s,ABC,ABCXYZ,score,total_rank
0,510003AMD.PL45,510003,AMD.PL45,Свеча зажигания (2015),AMD,Свеча зажигания 18855-10061/LZKR6B-10E/AMD.PL4...,193,24704.0,193,46.845001,0.0,1.0,X,24704.0,45.072998,1.0,A,AX,1.000000,1
15,56322AMD.PL33,56322,AMD.PL33,Свеча зажигания (2015),AMD,Свеча зажигания 96130723/BKR6E-11/AMD.PL33 (Ni...,44,5720.0,411,99.757004,0.0,16.0,X,5720.0,55.509003,2.0,A,AX,11.800000,14
14,56320AMD.PL11,56320,AMD.PL11,Свеча зажигания (2015),AMD,Свеча зажигания 18814-11051/BKR5ES-11/AMD.PL11...,61,4392.0,367,89.078003,0.0,15.0,X,4392.0,63.521999,3.0,A,AX,11.400000,12
1,515896AMD.PL51,515896,AMD.PL51,Свеча зажигания (2015),AMD,Свеча зажигания 7700500155/BKR-6EZ/AMD.PL51 (1...,41,4059.0,234,56.796001,0.0,2.0,X,4059.0,70.928001,4.0,A,AX,2.600000,2
9,523737AMD.PL111,523737,AMD.PL111,Свеча зажигания (2015),AMD,Свеча зажигания 22401-1KT1B/DILZKAR6A11/AMD.PL...,8,3752.0,290,70.388000,8.0,10.0,Y,3752.0,77.774002,5.0,A,AY,8.500000,8
2,515897AMD.PL52,515897,AMD.PL52,Свеча зажигания (2015),AMD,Свеча зажигания 7700500168/BKR-6EK/AMD.PL52 (2...,26,3640.0,260,63.106998,0.0,3.0,X,3640.0,84.415001,6.0,A,AX,3.900000,3
7,523731AMD.PL94,523731,AMD.PL94,Свеча зажигания (2015),AMD,Свеча зажигания 22401-1VA1C/DILKAR7D11H/AMD.PL...,4,1876.0,278,67.475998,4.0,8.0,Y,1876.0,87.837997,7.0,B,BY,7.700000,6
12,523786AMD.PL165,523786,AMD.PL165,Свеча зажигания (2015),AMD,Свеча зажигания A0031599703/IFR6D10/AMD.PL165 ...,4,1124.0,302,73.301003,4.0,13.0,Y,1124.0,89.889000,8.0,B,BY,11.500000,13
8,523735AMD.PL98,523735,AMD.PL98,Свеча зажигания (2015),AMD,Свеча зажигания 9807B-561-BW/IZFR6K11S/AMD.PL9...,4,1124.0,282,68.446999,4.0,9.0,Y,1124.0,91.939003,9.0,B,BY,9.000000,10
4,523719AMD.PL79,523719,AMD.PL79,Свеча зажигания (2015),AMD,Свеча зажигания 90919-01253/ILKAR7B11/AMD.PL79...,4,1124.0,266,64.563004,4.0,5.0,Y,1124.0,93.989998,10.0,B,BY,6.500000,4


In [39]:
e2.query('code1c == "508547"')

,uniq,code1c,oem,detal,brand,nomenclature,quantity,sales_x,cumsum_q,di_q,one_q,index_q,XYZ,sales_y,di_s,index_s,ABC,ABCXYZ,score,total_rank


In [40]:
fm.groupby('brand').agg({'quantity' : 'sum'})

,quantity
brand,
AMD,2585
FRANCECAR,4


In [41]:
brand = pd.concat([amd, fcr])

In [42]:
detal = pd.concat([fm, fv, fsp, fsu, fsa, ft, kf, kz, e2, e3, e4, e5])

In [43]:
detal.dtypes

uniq             object
code1c           object
oem              object
detal            object
brand            object
nomenclature     object
quantity          int32
sales_x         float32
cumsum_q          int32
di_q            float32
one_q           float32
index_q         float32
XYZ              object
sales_y         float32
di_s            float32
index_s         float32
ABC              object
ABCXYZ           object
score           float32
total_rank        int32
dtype: object

In [44]:
price['uniq'] = price.agg('{0[code1c]}{0[oem]}'.format, axis=1)
price['uniq'] = price['uniq'].replace(r'\s+', '', regex=True)

In [45]:
price_brand = price.merge(brand[['ABCXYZ', 'uniq', 'total_rank']], how='left', on='uniq')

In [46]:
price_brand

,code1c,oem,art,brand,group,detal,nomenclature,zn,comment,price,new,uniq,ABCXYZ,total_rank
0,525071,AMD.FC944A,04596501AB,AMD,Фильтр (2015),Фильтр салонный антибактериальный (2015),.Фильтр салонный 04596501AB/AMD.FC944A (антиба...,AMD / FCR 2018 (Склад),CHRYSLER (300 C LX) (2004-2010),"344,00",NaN,525071AMD.FC944A,NaN,NaN
1,525080,AMD.FC957A,05058693AA,AMD,Фильтр (2015),Фильтр салонный антибактериальный (2015),.Фильтр салонный 05058693AA/AMD.FC957A (антиба...,AMD / FCR 2018 (Склад),JEEP (Patriot/Liberty MK74) (2007-2017),"254,00",NaN,525080AMD.FC957A,NaN,NaN
2,519500,AMD.FC11A,0K2N16152X,AMD,Фильтр (2015),Фильтр салонный антибактериальный (2015),.Фильтр салонный 0K2N-161-52X/AMD.FC11A (антиб...,AMD / FCR 2018 (Склад),KIA (Spectra ИЖ) (Shuma FB),"463,00",16.08.2022 0:00:00,519500AMD.FC11A,NaN,NaN
3,525275,AMD.FC56A,0K55361C14A,AMD,Фильтр (2015),Фильтр салонный антибактериальный (2015),.Фильтр салонный 0K553-61-C14A/AMD.FC56A (анти...,AMD / FCR 2018 (Склад),"KIA (Carnival UP, GQ)","308,00",NaN,525275AMD.FC56A,NaN,NaN
4,525632,AMD.FC981A,1017016545,AMD,Фильтр (2015),Фильтр салонный антибактериальный (2015),.Фильтр салонный 1017016545/AMD.FC981A (антиба...,AMD / FCR 2018 (Склад),GEELY (Emgrand X7) (2018-) (CA),"220,00",NaN,525632AMD.FC981A,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4541,54258,AMD.FP159,311113K000,AMD,Электрика (2015),Электробензонасос (мотор) (2015),Электробензонасос 31111-3K000/AMD.FP159 AMD,AMD / FCR 2018 (Склад),HYUNDAI (Accent MC) (2005-) (Elantra HD) (2006...,"681,00",18.09.2013 0:00:00,54258AMD.FP159,AY,1201.0
4542,512468,AMD.FP212,311113L800,AMD,Электрика (2015),Электробензонасос (мотор) (2015),Электробензонасос 31111-3L800/AMD.FP212 AMD,AMD / FCR 2018 (Склад),"KIA (Optima TF) (2,0) (2,4) (2010-) (Soul AM) ...","506,00",05.06.2019 0:00:00,512468AMD.FP212,CZ,804.0
4543,509474,AMD.FP173,31119C9000,AMD,Электрика (2015),Электробензонасос (мотор) (2015),Электробензонасос 31119-C9000/AMD.FP173 AMD,AMD / FCR 2018 (Склад),HYUNDAI (Solaris II) (2017-) (Creta) KIA (Rio ...,"992,00",23.11.2018 0:00:00,509474AMD.FP173,AX,394.0
4544,515954,AMD.FP219,8200307403,AMD,Электрика (2015),Электробензонасос (мотор) (2015),Электробензонасос 8200307403/AMD.FP219 AMD,AMD / FCR 2018 (Склад),RENAULT (Logan LS) (2004-2015) (Sandero BS0) (...,"801,00",16.11.2020 0:00:00,515954AMD.FP219,NaN,NaN


In [47]:
price_brand_detal = price_brand.merge(detal[['ABCXYZ', 'uniq', 'total_rank']], how='left', on='uniq')

In [48]:
price_brand_detal

,code1c,oem,art,brand,group,detal,nomenclature,zn,comment,price,new,uniq,ABCXYZ_x,total_rank_x,ABCXYZ_y,total_rank_y
0,525071,AMD.FC944A,04596501AB,AMD,Фильтр (2015),Фильтр салонный антибактериальный (2015),.Фильтр салонный 04596501AB/AMD.FC944A (антиба...,AMD / FCR 2018 (Склад),CHRYSLER (300 C LX) (2004-2010),"344,00",NaN,525071AMD.FC944A,NaN,NaN,NaN,NaN
1,525080,AMD.FC957A,05058693AA,AMD,Фильтр (2015),Фильтр салонный антибактериальный (2015),.Фильтр салонный 05058693AA/AMD.FC957A (антиба...,AMD / FCR 2018 (Склад),JEEP (Patriot/Liberty MK74) (2007-2017),"254,00",NaN,525080AMD.FC957A,NaN,NaN,NaN,NaN
2,519500,AMD.FC11A,0K2N16152X,AMD,Фильтр (2015),Фильтр салонный антибактериальный (2015),.Фильтр салонный 0K2N-161-52X/AMD.FC11A (антиб...,AMD / FCR 2018 (Склад),KIA (Spectra ИЖ) (Shuma FB),"463,00",16.08.2022 0:00:00,519500AMD.FC11A,NaN,NaN,NaN,NaN
3,525275,AMD.FC56A,0K55361C14A,AMD,Фильтр (2015),Фильтр салонный антибактериальный (2015),.Фильтр салонный 0K553-61-C14A/AMD.FC56A (анти...,AMD / FCR 2018 (Склад),"KIA (Carnival UP, GQ)","308,00",NaN,525275AMD.FC56A,NaN,NaN,NaN,NaN
4,525632,AMD.FC981A,1017016545,AMD,Фильтр (2015),Фильтр салонный антибактериальный (2015),.Фильтр салонный 1017016545/AMD.FC981A (антиба...,AMD / FCR 2018 (Склад),GEELY (Emgrand X7) (2018-) (CA),"220,00",NaN,525632AMD.FC981A,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4541,54258,AMD.FP159,311113K000,AMD,Электрика (2015),Электробензонасос (мотор) (2015),Электробензонасос 31111-3K000/AMD.FP159 AMD,AMD / FCR 2018 (Склад),HYUNDAI (Accent MC) (2005-) (Elantra HD) (2006...,"681,00",18.09.2013 0:00:00,54258AMD.FP159,AY,1201.0,AX,38.0
4542,512468,AMD.FP212,311113L800,AMD,Электрика (2015),Электробензонасос (мотор) (2015),Электробензонасос 31111-3L800/AMD.FP212 AMD,AMD / FCR 2018 (Склад),"KIA (Optima TF) (2,0) (2,4) (2010-) (Soul AM) ...","506,00",05.06.2019 0:00:00,512468AMD.FP212,CZ,804.0,CZ,25.0
4543,509474,AMD.FP173,31119C9000,AMD,Электрика (2015),Электробензонасос (мотор) (2015),Электробензонасос 31119-C9000/AMD.FP173 AMD,AMD / FCR 2018 (Склад),HYUNDAI (Solaris II) (2017-) (Creta) KIA (Rio ...,"992,00",23.11.2018 0:00:00,509474AMD.FP173,AX,394.0,AX,3.0
4544,515954,AMD.FP219,8200307403,AMD,Электрика (2015),Электробензонасос (мотор) (2015),Электробензонасос 8200307403/AMD.FP219 AMD,AMD / FCR 2018 (Склад),RENAULT (Logan LS) (2004-2015) (Sandero BS0) (...,"801,00",16.11.2020 0:00:00,515954AMD.FP219,NaN,NaN,NaN,NaN


In [49]:
price_brand_detal['new'].loc[price_brand_detal.new.str.contains("2023", na=False)] = "Новинка"

In [50]:
#Удаление всех значений кроме Новика 2022 и Новинка 2023
price_brand_detal['new'].loc[price_brand_detal.new.str.contains("00", na=False)] = np.nan

In [51]:
price_brand_detal['ABCXYZ_y'] = price_brand_detal['ABCXYZ_y'].replace(np.nan,'ZZ')
price_brand_detal['ABCXYZ_x'] = price_brand_detal['ABCXYZ_x'].replace(np.nan,'ZZ')

In [52]:
price_brand_detal['total_rank_y'] = price_brand_detal['total_rank_y'].replace(np.nan, 0)
price_brand_detal['total_rank_x'] = price_brand_detal['total_rank_x'].replace(np.nan, 0)

In [53]:
price_brand_detal['max_total_rank'] = np.where((price_brand_detal['total_rank_x'] > price_brand_detal['total_rank_y'])
                                              , price_brand_detal['total_rank_x'], price_brand_detal['total_rank_y'])

In [54]:
price_brand_detal['ABCXYZ_y'] = price_brand_detal['ABCXYZ_y'].replace(['AX', 'BX', 'AY', 'BY', 'AZ', 'BZ', 'CX', 'CY', 'CZ', 'ZZ'],
                                                                      [9, 8, 7, 6, 5, 4, 3, 2, 1, 0])
price_brand_detal['ABCXYZ_x'] = price_brand_detal['ABCXYZ_x'].replace(['AX', 'BX', 'AY', 'BY', 'AZ', 'BZ', 'CX', 'CY', 'CZ', 'ZZ'],
                                                                      [9, 8, 7, 6, 5, 4, 3, 2, 1, 0])


In [55]:
price_brand_detal['TOTAL_ABCXYZ_'] = np.where((price_brand_detal['ABCXYZ_y'] < price_brand_detal['ABCXYZ_x'])
                                                             , price_brand_detal['ABCXYZ_x'], price_brand_detal['ABCXYZ_y'])

In [56]:
count_rank = price_brand_detal.code1c.count()

In [57]:
price_brand_detal['max_total_rank'] = price_brand_detal.max_total_rank.where(
    price_brand_detal['max_total_rank'] != 0, count_rank + 1)

In [58]:
price_brand_detal['ABCXYZ_y'] = price_brand_detal['ABCXYZ_y'].replace([9, 8, 7, 6, 5, 4, 3, 2, 1, 0], 
                                                                     ['AX', 'BX', 'AY', 'BY', 'AZ', 'BZ', 'CX', 'CY', 'CZ', ''])
price_brand_detal['ABCXYZ_x'] = price_brand_detal['ABCXYZ_x'].replace([9, 8, 7, 6, 5, 4, 3, 2, 1, 0], 
                                                                     ['AX', 'BX', 'AY', 'BY', 'AZ', 'BZ', 'CX', 'CY', 'CZ', ''])
price_brand_detal['TOTAL_ABCXYZ_'] = price_brand_detal['TOTAL_ABCXYZ_'].replace([9, 8, 7, 6, 5, 4, 3, 2, 1, 0], 
                                                                     ['AX', 'BX', 'AY', 'BY', 'AZ', 'BZ', 'CX', 'CY', 'CZ', 'CZ'])

In [59]:
price_brand_detal['total_rank_x'] = price_brand_detal['total_rank_x'].astype(int)
price_brand_detal['total_rank_y'] = price_brand_detal['total_rank_y'].astype(int)
price_brand_detal['max_total_rank'] = price_brand_detal['max_total_rank'].astype(int)

In [60]:
price_brand_detal = price_brand_detal.rename(columns={'code1c' : 'id', 
                                                     'oem' : 'OEM',
                                                     'art' : 'Артикул',
                                                     'brand' : 'Бренд',
                                                     'group' : 'Группа',
                                                     'detal' : 'Деталь',
                                                     'nomenclature' : 'Номенклатура',
                                                     'zn' : 'З/Н',
                                                     'comment' : 'Комментарий',
                                                     'price' : 'Цена',
                                                     'new' : 'Новинка',
                                                     'uniq' : 'Уникльный ключ',
                                                     'ABCXYZ_x' : 'Категория по бренду',
                                                     'total_rank_x' : 'Ранг по бренду',
                                                     'ABCXYZ_y' : 'Категория по деталям',
                                                     'total_rank_y' : 'Ранг по деталям',
                                                     'max_total_rank' : 'Ранг',
                                                     'TOTAL_ABCXYZ_' : 'abc/xyz'})

In [61]:
price_brand_detal['Дата'] = '01.08.2023'

In [62]:
price_brand_detal['id'] = price_brand_detal['id'].astype('object')

In [63]:
price_brand_detal['Компания'] = 'Москворечье'

In [64]:
price_brand_detal.to_csv('08_2023_abcxyz_moskvorechie.txt', sep=';', encoding='UTF-8', index=False)